In [6]:

import tensorflow as tf


# ROOT = os.getcwd()
# What model to download.
MODEL_PATH = 'Model/800img2.pb'
#Path to input images
PATH_TO_TEST_IMAGES = 'output/'

# List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = os.path.join('training', 'labelmap.pbtxt')

OUTPUT_PATH = 'output/'
threshold = 0.1

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(MODEL_PATH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
print('done')

done


In [15]:
def run_inference_for_multiple_image(images, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      for image_path in images:
          print(image_path)
          image = Image.open(image_path)
          
          #Run inference
          image_np = load_image_into_numpy_array(image)
          image_np_expanded = np.expand_dims(image, axis=0)
          output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

#           print(output_dict['detection_scores'])
          # all outputs are float32 numpy arrays, so convert types as appropriate
          output_dict['num_detections'] = int(output_dict['num_detections'][0])
          output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
          output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
          output_dict['detection_scores'] = output_dict['detection_scores'][0]
          if 'detection_masks' in output_dict:
            output_dict['detection_masks'] = output_dict['detection_masks'][0]
          count = 0
          for box,classes,score in zip(output_dict['detection_boxes'],output_dict['detection_classes'],output_dict['detection_scores']):
            #Drawing boxes takes a long time!!!
            if score >1.0:
                ymin,xmin,ymax,xmax = box
                xmin,xmax= converse(xmin,xmax,image.size[0])
                ymin,ymax= converse(ymin,ymax,image.size[1])
                image = draw_rect(np.array(image),image_path,xmin,ymin,xmax,ymax,score,(255,255,255))
      #saver.save(sess,graphOutputTest)
  return output_dict
print('done')


done


In [11]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [14]:
import numpy as np
from PIL import Image
output_dict = run_inference_for_multiple_image(['eval/test/hori00007.png'],detection_graph)

eval/test/hori00007.png
